# Rating-Aware Beer 🍺 Recommendation System

This is an intelligent beer recommendation system that combines **predictive quality assessment** with **similarity-based recommendations** to provide both what users want and educational guidance about better alternatives.

Simply give a prompt describing what you are craving, and the system will:

- Predict how well the beer flavor profile rates.
- Provide a comprehensive recommendation for that flavor.

But that's not all! If the flavor profile rates poorly, the system will:

- Warn the user.
- Suggest other similar-flavored, high-rated beers.

Enjoy smarter beer choices tailored just for you!

## System Architecture

The solution or pipeline consists of four integrated components:

1. **Natural Language Processing Pipeline**
   - Converts user queries like "light citrusy beer" into numerical flavor profiles
   - Uses Groq's Llama 3.1 model with structured prompting
   - Handles 12 flavor dimensions + style classification

2. **Rating Prediction Component**
   - Gradient Boosted Trees model predicts beer quality (1-5 stars)
   - Trained on flavor profiles to predict user satisfaction
   - Acts as quality gate for recommendations

3. **Recommendation Engine**
   - K-Nearest Neighbors finds similar beers based on flavor profiles
   - Content-based filtering by mainstream/craft and alcohol strength
   - Bayesian quality scoring incorporating review volume

4. **Intelligent User Interface**
   - Warns users about potentially disappointing flavor combinations
   - When predicted rating < 3.0/5, alerts users about potentially disappointing choices
   - Provides exact matches for user requests
   - Suggests higher-rated alternatives when needed
   - Educational messaging about flavor compatibility

## Dataset Used

The comprehensive [Beer Profile and Ratings Dataset](https://www.kaggle.com/datasets/ruthgn/beer-profile-and-ratings-data-set) containing 33,000+ beer reviews with detailed flavor profiles.

## Flow chart

![Alt text](flowchart.png)

## Data Preprocessing and Feature Engineering

* Removed unneeded columns (IBU ranges, individual review components, name fields)
* Added 'mainstream' binary feature to classify mass-market vs craft beers
* Created 'strength' categorical feature based on ABV ranges (Light/Medium/Strong/Extra Strong)

In [1]:
import pandas as pd
import numpy as np
import re

# Preprocessing
df = pd.read_csv('./data/beer_profile_and_ratings.csv')


# List of mainstream beer patterns
mainstream_patterns = [
     
    # company names
    'co.', 'inc'
    # Major American Brands
    'budweiser', 'bud', 'busch', 'michelob',
    'miller', 'coors', 'keystone', 'blue moon',
    'pabst', 'pbr', 'schlitz', 'old milwaukee',
    'rolling rock', 'yuengling', 'natural light', 'natty',
    
    # Sam Adams
    'samuel adams', 'sam adams', 'boston lager',
    
    # Mexican/Latin Beers
    'corona', 'modelo', 'pacifico',
    'dos equis', 'tecate', 'sol', 'victoria',
    
    # European Imports
    'heineken', 'amstel', 'stella artois',
    'becks', "beck's", 'st pauli', 'warsteiner',
    'guinness', 'harp', 'smithwick', 'kilkenny',
    'peroni', 'moretti', 'nastro azzurro',
    'carlsberg', 'tuborg', 'kronenbourg',
    'fosters', "foster's", 'grolsch', 'pilsner urquell',
    
    # Canadian
    'molson', 'labatt', 'moosehead', 'sleeman',
    
    # Asian
    'sapporo', 'asahi', 'kirin', 'tsingtao', 'singha', 'tiger', 'leo',
    
    # Large Craft (Now Owned by Big Beer)
    'shock top', 'goose island', 'elysian', 'lagunitas',
    'ballast point', '10 barrel', 'golden road',
    'blue point', 'devils backbone', 'karbach',
    'breckenridge', 'four peaks', 'wicked weed',
    
    # Large Independent Craft (Widely Distributed)
    'sierra nevada', 'new belgium', 'fat tire',
    'stone', 'brooklyn', 'dogfish head', 
    "bell's", 'bells brewery', 'founders',
    'deschutes', 'rogue', 'anchor steam',
    
    # Other Mainstream
    'red stripe', 'newcastle', 'bass', 'boddingtons',
    'murphy', 'beamish', 'tennents', 'carling',
    'leinenkugel', 'magic hat', 'pyramid',
    'widmer', 'redhook', 'kona', 'longboard',
    'landshark', 'presidente', 'medalla',
    
    # Indian Beers
    'kingfisher', 'haywards', 'thunderbolt',
    'kalyani', 'knockout', 'royal challenge',
    'carlsberg elephant', 'bira 91', 'bira',
    'simba', 'godfather', 'hunter', 'zingaro',
    'london pilsner', 'kotsberg', 'bullet',
    'khajuraho', 'taj mahal', 'flying horse', 'dansberg',
    'golden eagle', 'guru', 'bad monkey', 'bee young',
    'white rhino', 'white owl', 'effingut'
]


# marking which beers are from mainstream brands (off the shelf)
def matches_mainstream_pattern(beer_name_full):
        """Check if beer/brewery name matches any mainstream pattern"""
        combined_name = beer_name_full.lower()
        
        for pattern in mainstream_patterns:
            if pattern in combined_name:
                return True
        return False

df['mainstream'] = df.apply(lambda row : matches_mainstream_pattern(row['Beer Name (Full)']), axis=1)


df['mainstream'] = df['mainstream'] | (df['number_of_reviews'] >= 300)

df['strength'] = df['ABV'].apply(
    lambda x: 'Light' if x <= 5 else
              'Medium' if x <= 7 else
              'Strong' if x <= 10 else
              'Extra Strong'
)

df = df.drop(columns=['Min IBU', 'Max IBU', 'review_aroma', 'review_appearance', 'review_palate', 'review_taste', 'Beer Name (Full)', 'Brewery'])


cols = df.columns.tolist()
cols[1], cols[2] = cols[2], cols[1]
df = df[cols]
df['mainstream'] = df['mainstream'].astype(int)

print("Preleminary Data Preprocessing completed.")


Preleminary Data Preprocessing completed.


## Regression Component

* **Data preparation**: Applied one-hot encoding for categorical features and min-max scaling for numerical features
* **Model comparison**: Evaluated Linear Regression vs Gradient Boosted Trees (GBT  outperformed with lower MSE)
* **Final model**: Trained Gradient Boosted Trees with optimized hyperparameters (`n_estimators=150, learning_rate=0.1, max_depth=4`)

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder
import re

# Preprocessing and feature Engineering for Regression

reg_df = df.drop(columns=['number_of_reviews', 'strength', 'Name', 'Description'])

cols = reg_df.columns.tolist()
# Swap last two
cols[-2], cols[-1] = cols[-1], cols[-2]
# Reorder dataframe
reg_df = reg_df[cols]


y_reg = reg_df.iloc[:, -1]
X_reg = reg_df.iloc[:, :-1]

X = X_reg.copy()

flavor_features = ['ABV', 'Astringency', 'Body', 'Alcohol', 'Bitter', 'Sweet', 'Sour', 'Salty', 'Fruits', 'Hoppy', 'Spices', 'Malty']

scalar = MinMaxScaler()
# Create global scaler fitted on ALL data
scalar.fit(df[flavor_features])
# Then in your recommendation function, Only TRANSFORM, don't fit
X[flavor_features] = scalar.transform(X[flavor_features])


# Removing sub categories like Lager-English, Lager-belgium to just Lager
# Using regex to split by ' - ' first, then by ' / ' if no hyphen
X['Style'] = X['Style'].str.split(' - ').str[0].str.split(' / ').str[0]

# Create encoder
encoder = OneHotEncoder(sparse_output=False)

# Fit and transform the 'style' column
encoded_array = encoder.fit_transform(X[['Style']])

# Get feature names
feature_names = encoder.get_feature_names_out(['Style'])

# Create DataFrame with encoded features
encoded_df = pd.DataFrame(encoded_array, columns=feature_names, index=X_reg.index)

# Concatenate with original DataFrame (dropping the original 'style' column)
X_reg_scaled = pd.concat([X.drop('Style', axis=1), encoded_df], axis=1)

print("Dataset Preparation for Regression Component Completed")


Dataset Preparation for Regression Component Completed


In [3]:
# Testing Various Regression Models

from sklearn.linear_model import LinearRegression, Ridge
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.neural_network import MLPRegressor

X_reg_np = X_reg_scaled.to_numpy()
y_reg_np = y_reg.to_numpy()

X_train, X_test, y_train, y_test = train_test_split(X_reg_np, y_reg_np, test_size=0.2)

linear_model = LinearRegression()
linear_model.fit(X_train, y_train)

y_pred_linear = linear_model.predict(X_test)

linear_mse = mean_squared_error(y_pred_linear, y_test)

print(f"Test MSE for linear model {linear_mse}")

 #Train Gradient Boosting

gb_model = GradientBoostingRegressor(
    n_estimators=150,
    learning_rate=0.1,
    max_depth=4,
)

gb_model.fit(X_train, y_train)
y_pred_gb = gb_model.predict(X_test)

# Calculate MSE
gb_mse = mean_squared_error(y_pred_gb, y_test)

print(f"Test MSE for Gradient Boosted trees model {gb_mse}")
print(f"diff = {linear_mse - gb_mse}\n")


print("Gradient Boosted trees for the win")


Test MSE for linear model 0.11848367728249891
Test MSE for Gradient Boosted trees model 0.10425300239136617
diff = 0.014230674891132736

Gradient Boosted trees for the win


In [4]:
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import GradientBoostingRegressor

gb_model = GradientBoostingRegressor(
    n_estimators=150,
    learning_rate=0.1,
    max_depth=4)

# Training on the whole dataset
X_train = X_reg_scaled.to_numpy()
y_train = y_reg.to_numpy()

gb_model.fit(X_train, y_train)

print("Gradient Boosted trees for rating Prediction Ready")


Gradient Boosted trees for rating Prediction Ready


## LLM Wrapper


* **Natural Language Processing**: Implemented LLM wrapper using Groq's Llama 3.1 model with custom system prompt
* **Feature Translation**: Converts natural language beer preferences into structured numerical flavor profiles
* **Intelligent Parsing**: Handles complex descriptors like "light citrusy" and maps them to appropriate feature values

    **Example Transformation:**

    Input: "I want a light citrusy beer"

    ↓ LLM Wrapper ↓

    Output:
    ```json
    {
    'ABV': 4.2,
    'Astringency': 12,
    'Body': 35,
    'Alcohol': 15,
    'Bitter': 45,
    'Sweet': 40,
    'Sour': 85,
    'Salty': 0,
    'Fruits': 140,
    'Hoppy': 65,
    'Spices': 15,
    'Malty': 50,
    'mainstream': 1,
    'style': 'Wheat Beer'
    }
    ```



In [5]:
import os
from groq import Groq
from dotenv import load_dotenv
import json

load_dotenv()

def get_beer_features_from_text(user_input):
    """Convert natural language beer preference to features using GROQ"""
    try: 
        client = Groq(api_key=os.getenv("GROQ_API_KEY"))
        
        # The prompt from your paste.txt file
        system_prompt = """
        You are a beer flavor profile translator. Convert natural language beer preferences into numerical flavor profiles.

        ## Output Format
        Return a JSON with these exact fields:
        - ABV: (float) 0.0-57.5
        - Astringency: (int) 0-81
        - Body: (int) 0-175
        - Alcohol: (int) 0-139
        - Bitter: (int) 0-150
        - Sweet: (int) 0-263
        - Sour: (int) 0-284
        - Salty: (int) 0-48
        - Fruits: (int) 0-175
        - Hoppy: (int) 0-172
        - Spices: (int) 0-184
        - Malty: (int) 0-239
        - mainstream: (int) 0 or 1 (DEFAULT = 1)
        - style: (string) Beer style category

        ## IMPORTANT: Mainstream Flag Rules
        DEFAULT mainstream = 1 (always start with 1)

        Only set mainstream = 0 when:
        - Belgian styles mentioned (Tripel, Dubbel, Quad)
        - Sour/Wild/Lambic/Brett explicitly mentioned
        - Imperial/Dessert beers with ABV > 9
        - User explicitly says "craft", "artisanal", "specialty", "drought", "non mainstream"
        - Highly experimental flavor combinations

        Keep mainstream = 1 for:
        - All standard styles (IPA, Pilsner, Lager, Wheat, Stout, Amber)
        - Any request without special keywords above
        - "Sessionable", "refreshing", "light" beers
        - When in doubt, use mainstream = 1

        ## Scaling Guidelines
        Use percentages of max range:
        - "Very low/minimal": 3-10%
        - "Low/light": 10-25%
        - "Moderate/medium": 25-45%
        - "High": 50-70%
        - "Very high": 70-85%
        - "Extremely/maximum": 85-100%

        ## Core Translation Rules

        ### Intensity Modifiers
        - No modifier = use style default or 30-50% range
        - "Slightly/hint of" = reduce by 50%
        - "Very" = 70-85% of max
        - "Extremely/super" = 85-100% of max
        - "No/without" = 5-10% of max

        ### Strength/Alcohol Keywords
        - "light" → ABV: 3.2-4.5, Body: 25-35 (15-20%), Alcohol: 10-20 (7-14%)
        - "sessionable" → ABV: 4-5, Body: 30-40 (17-23%), Alcohol: 15-25
        - "medium/regular" → ABV: 5-6, Body: 60-80 (34-46%), Alcohol: 40-70
        - "strong" → ABV: 7-9, Body: 70-90, Alcohol: 75-100 (54-72%)
        - "very strong/imperial" → ABV: 9-12, Body: 120-160, Alcohol: 100-130

        ### Flavor Keywords
        - "citrusy" → Fruits: 140 (80%), Sour: 85 (30%)
        - "tropical" → Fruits: 145 (83%), Sour: 15 (5%)
        - "orangey" → Fruits: 155 (89%), add Sour: 240 if "tart"
        - "fruity" → Fruits: 120 (69%)
        - "chocolate" → Spices: 140 (76%), Malty: 210 (88%)
        - "coffee" → Spices: 140 (76%), Astringency: 55 (68%)
        - "spicy" → Spices: 155 (84%)
        - "funky/brett" → Sour: 265 (93%), Astringency: 65 (80%)
        - "tart" → Sour: 240+ (85%+), Astringency: 45+ (56%+)

        ### Hop/Bitter Keywords
        - "hoppy" → Hoppy: 150 (87%), Bitter: 110 (73%)
        - "very hoppy" → Hoppy: 155-165 (90-96%), Bitter: 120-135
        - "bitter" → Bitter: 110-135 (73-90%)
        - "no hops" → Hoppy: 20 (12%), Bitter: 20 (13%)

        ### Sweet/Malty Keywords
        - "sweet" → Sweet: 145 (55%)
        - "very sweet" → Sweet: 195-210 (74-80%)
        - "dessert" → Sweet: 195 (74%), Body: 160 (91%)
        - "no sweetness/dry" → Sweet: 15 (6%)
        - "malty" → Malty: 185 (77%)
        - "very malty" → Malty: 210 (88%)
        - "not too malty" → Malty: 60 (25%)

        ## Style Templates

        ### IPA (mainstream = 1)
        Base: Hoppy: 155, Bitter: 110, ABV: 6.8, Body: 75, Malty: 75

        ### Pilsner (mainstream = 1)
        Base: Hoppy: 65, Bitter: 45, ABV: 4.5, Body: 30, Malty: 80

        ### Wheat Beer (mainstream = 1)
        Base: Hoppy: 45, Body: 35, ABV: 4.2, Fruits: 85, Sour: 65

        ### Lager (mainstream = 1)
        Base: Hoppy: 60, Bitter: 55, ABV: 5.0, Body: 50, Malty: 60

        ### Stout (mainstream = 1 unless imperial)
        Base: Body: 140, Malty: 180, ABV: 6.5, Hoppy: 35
        Imperial: ABV: 10.5, Body: 160, mainstream = 0

        ### Belgian Tripel (mainstream = 0)
        Base: ABV: 9.0, Spices: 155, Fruits: 95, Sweet: 115

        ### Sour/Wild Ale (mainstream = 0)
        Base: Sour: 265, Astringency: 65, Hoppy: 20, mainstream = 0

        ### Amber/Red Ale (mainstream = 1)
        Base: Malty: 185, Sweet: 145, Body: 95, Bitter: 35

        ### Light Beer (mainstream = 1)
        Base: ABV: 3.2, Body: 25, all others low (10-30% range)

        ## Processing Order
        1. Identify style first (sets base template)
        2. Apply strength modifiers (light/strong/sessionable)
        3. Apply flavor descriptors (additive)
        4. Apply negations last (no sweetness, etc.)
        5. Check mainstream flag (default = 1 unless special style)

        ## Examples
        "hoppy IPA" → Start with IPA template, already has high hoppy
        "light beer" → Use Light Beer template
        "Belgian tripel" → Use Tripel template, set mainstream = 0
        "dessert stout" → Stout template + high sweet/body, mainstream = 0

        ## Special Edge Cases

        ### Explicitly Bad/Poor Quality Requests
        When user explicitly asks for "bad", "terrible", "awful", "worst", "horrible", "disgusting", "undrinkable" beer:
        - Set ALL features to minimum values (3-10% of max)
        - ABV: 0.05-1.0
        - All flavor features: 1-10% of their max values
        - Astringency: 2-5
        - Body: 10-15
        - Alcohol: 10-15
        - Bitter: 3-10
        - Sweet: 10-20
        - Sour: 3-10
        - Salty: 0-2
        - Fruits: 1-10
        - Hoppy: 3-10
        - Spices: 3-10
        - Malty: 15-25
        - mainstream: 1
        - style: "Low Alcohol Beer" or "Light Beer"

        Examples:
        - "Just a bad beer" → Minimal everything
        - "Give me your worst beer" → Lowest possible values
        - "I want a terrible beer" → Near-zero features
        - "Something awful" → Minimum profile

        ### Testing/Experimental Requests
        If user mentions "test", "experiment", or asks for unusual combinations that would clearly conflict (e.g., "extremely sweet AND extremely bitter AND light body"), recognize this as potentially problematic and generate values that reflect the conflict.
                        
                        """
        
        response = client.chat.completions.create(
            model="llama-3.1-8b-instant",  # Free tier model
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_input}
            ],
            temperature=0.3,  # Lower for more consistent outputs
            response_format={"type": "json_object"}  # Force JSON output
        )
        
        return json.loads(response.choices[0].message.content)
    except Exception as e:
        print(f"Error calling GROQ API: {e}")

print("Llama LLM Wrapper Initialized")

Llama LLM Wrapper Initialized


## Recommendation Component

* **Test Point Generation**: Takes LLM output and creates test datapoint with proper scaling and encoding
* **Content-Based Filtering**: Subsets dataset according to hard filters (mainstream/craft preference, alcohol strength categories)
* **Similarity Search**: Uses K-Nearest Neighbors with configuration `NearestNeighbors(n_neighbors=10, metric='euclidean')`
* **Custom Quality Scoring**: Applies custom Bayesian averaging formula incorporating review volume for final ranking
* **Rating-Aware Display**: 
 - If predicted rating ≥ 3.0: Shows standard recommendations with positive messaging
 - If predicted rating < 3.0: Displays warning about flavor combination, provides exact matches, and suggests better-rated alternatives

**Example Outputs:**

**Good Combination (Rating ≥ 3.0):**

```
 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
✅ Great choice! Predicted rating: 3.75/5
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

🍺 Top Recommendations:

1. Sunshine Pils
   Rating: 4.06/5 (533 reviews)
   Distance: 0.878
   Notes: Notes:Like the rising sun, Sunshine Pils delivers winter, spring, summer and fall. This deceptive complex pilsner is all...

2. Pikeland Pils
   Rating: 4.14/5 (318 reviews)
   Distance: 0.902
   Notes: Notes:OG 11 ºP / 44 IBUs\t...

3. Scrimshaw Pilsner
   Rating: 3.93/5 (576 reviews)
   Distance: 0.828
   Notes: Notes:Named for the delicate engravings popularized by 19th century seafarers, Scrimshaw is a fresh tasting Pilsner brew...
```

**Problematic Combination (Rating < 3.0):**

```
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
⚠️  Warning: This flavor combination typically rates 2.90/5
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

📍 Here's what matches your exact request:
1. The Kaiser (3.65★ - 581 reviews)
   Distance: 0.903
2. Hevelius Kaper (3.32★ - 83 reviews)
   Distance: 0.830
3. Labatt Max Ice (3.11★ - 60 reviews)
   Distance: 0.859

💡 Suggested Alternatives (similar but better rated):
1. Abt 12 (4.33★ - 2217 reviews)
   Distance: 1.546
2. Three Philosophers (3.99★ - 1683 reviews)
   Distance: 1.546
3. The Kaiser (3.65★ - 581 reviews)
   Distance: 0.903

💭 Tip: The flavor combination you requested is uncommon. The alternatives above
   maintain similar characteristics but with proven appeal to beer enthusiasts.

────────────────────────────────────────────────────────────
```

In [6]:
def get_strength(ABV):
    
  if ABV <= 5:
    strength = 'Light'
  elif ABV <= 7:
    strength = 'Medium'
  elif ABV <= 10:
    strength = 'Strong'
  else:
    strength = 'Extra Strong'

  return strength


scaling_features = ['ABV', 'Astringency', 'Body', 'Alcohol', 'Bitter','Sweet','Sour', 'Salty',	'Fruits',	'Hoppy'	,'Spices',	'Malty']

X_recommend = df[['Style'] + scaling_features + ['mainstream','strength']].copy()

y_recommend = df[['Name', 'Description', 'review_overall', 'number_of_reviews']]

X_recommend['Style'] = X_recommend['Style'].str.split(' - ').str[0].str.split(' / ').str[0]

# Create encoder
encoder2 = OneHotEncoder(sparse_output=False)

# Fit and transform the 'style' column
encoded_array = encoder2.fit_transform(X_recommend[['Style']])

# Get feature names
feature_names = encoder2.get_feature_names_out(['Style'])

# Create DataFrame with encoded features
encoded_df = pd.DataFrame(encoded_array, columns=feature_names, index=X_recommend.index)

# Concatenate with original DataFrame (dropping the original 'style' column)
X_recommend = pd.concat([X_recommend.drop('Style', axis=1), encoded_df], axis=1)

print("Dataset Prepared for Recommendation Component")

Dataset Prepared for Recommendation Component


In [7]:
from sklearn.neighbors import NearestNeighbors
import numpy as np


def get_quality_score(rating, num_reviews):

    # Calculates final quality score using Bayseian Average (IMdb)
    # Incorporates num_reviews in final score
    m = 50  
    C = 3.748
    return rating * (0.6 + 0.4 * np.log1p(num_reviews) / 10)


# Initialize KNN model
knn = NearestNeighbors(n_neighbors=10, metric='euclidean')

print("knn with 10 neighbours initialized")

knn with 10 neighbours initialized


# Completed workflow/Pipeline

Following cell runs Example Prompts for testing

To try out your own prompt simply add 

```
<"your prompt">: get_beer_features_from_text(<"your prompt">)
```
in the ``` test_cases``` dictionary

In [8]:
# Testing

import pandas as pd

# Show all columns
pd.set_option('display.max_columns', None)

# Show all rows
pd.set_option('display.max_rows', None)

# Avoid line-wrapping or truncated cells
pd.set_option('display.max_colwidth', None)

def generate_test_point(llm_output, X_train, scalar, type):

    scaling_features = ['ABV', 'Astringency', 'Body', 'Alcohol', 'Bitter', 
                      'Sweet', 'Sour', 'Salty', 'Fruits', 'Hoppy', 'Spices', 'Malty']


    test_point = {col: 0 for col in X_train.columns}

    # Fill in the scaled continuous features
    for feat in scaling_features:
        test_point[feat] = [llm_output[feat]]

    # One-hot encode the style
    style_column = f"Style_{llm_output['style']}"
    if style_column in X_train.columns:
        test_point[style_column] = 1
    
    if type == 'Regressor':
        # Set mainstream (doesn't need scaling)
        test_point['mainstream'] = llm_output['mainstream']

    test_point = pd.DataFrame(test_point)

    test_point[scaling_features] = scalar.transform(test_point[scaling_features])
    # Make sure Columns are in same order of the training set
    test_point = test_point[X_train.columns] 

    return test_point



def get_beer_recommendations(llm_output, X_recommend, y_recommend, alt = False, alt_rating_threshold = 3.5, top_n = 3):

    # PERFORMING RECOMMENDATION
    # subsetting the dataframe for proper recommendation

    # for alternative recommendations rating is given preference
    if alt:
        rating_mask = y_recommend['review_overall'] >= alt_rating_threshold

        X_recommend = X_recommend[rating_mask]
        y_recommend = y_recommend[rating_mask] 

    if llm_output['mainstream'] == 1: # filter according to mainstream

        mainstream_mask = X_recommend['mainstream'] == 1

        X_recommend = X_recommend[mainstream_mask]
        y_recommend = y_recommend[mainstream_mask]

    strength = get_strength(llm_output['ABV'])

    strength_mask = X_recommend['strength'] == strength

    X_recommend_sub = X_recommend[strength_mask]
    y_recommend_sub = y_recommend[strength_mask]


    # Dropping strength and mainstream used for content based filtering
    X_recommend_sub = X_recommend_sub.drop(columns=['strength', 'mainstream'])


    # Scaling after subsetting to reduce noise
    scalar2 = MinMaxScaler()
    # Create global scaler fitted on ALL data
    scalar2.fit(df[scaling_features])
    # Then in your recommendation function, Only TRANSFORM, don't fit
    X_recommend_sub[scaling_features] = scalar2.transform(X_recommend_sub[scaling_features])


    # display(X_recommend.columns)

    X_recommend_scaled = X_recommend_sub

    X_recommend_scaled_np = X_recommend_scaled.to_numpy()
    y_recommend_np = y_recommend_sub.to_numpy()

    test_point_recommendation = generate_test_point(llm_output, X_recommend_scaled, scalar2, type="Recommend")

    test_point_recommendation_np = test_point_recommendation.values[0]

    # display(test_point_recommendation)

    knn.fit(X_recommend_scaled_np)  # Included in Loop because subsetting always changes the df

    # Find 10 nearest neighbors
    distances, indices = knn.kneighbors([test_point_recommendation_np])

    # Get the beer info for these 10 neighbors
    top_10_beers = []
    for i, idx in enumerate(indices[0]):
        beer_info = {
            'name': y_recommend_np[idx][0],
            'description': y_recommend_np[idx][1],
            'rating': y_recommend_np[idx][2],
            'num_reviews': y_recommend_np[idx][3],
            'distance': distances[0][i],
            'index': idx
        }
        top_10_beers.append(beer_info)


    # Create quality score 
    for beer in top_10_beers:
        beer['quality_score'] = get_quality_score(beer['rating'], beer['num_reviews'])

        
    # Sort by quality score (descending)
    top_10_beers.sort(key=lambda x: x['quality_score'], reverse=True)

    # Get top 2
    final_recommendations = top_10_beers[:top_n]

    return final_recommendations



def display_results(predicted_rating, regular_recommendations, alt_recommendations=None):
    """
    Display recommendations with warnings for low-rated combinations
    
    """
    
    if alt_recommendations is not None:
        # Low rating case - show warning and alternatives
        print("━" * 60)
        print(f"⚠️  Warning: This flavor combination typically rates {predicted_rating:.2f}/5")
        print("━" * 60)
        
        print("\n📍 Here's what matches your exact request:")
        if regular_recommendations:
            for i, beer in enumerate(regular_recommendations[:3], 1):
                print(f"{i}. {beer['name']} ({beer['rating']:.2f}★ - {beer['num_reviews']} reviews)")
                print(f"   Distance: {beer['distance']:.3f}")
        else:
            print("   No exact matches found in our database.")
        
        print("\n💡 Suggested Alternatives (similar but better rated):")
        if alt_recommendations:
            for i, beer in enumerate(alt_recommendations[:3], 1):
                print(f"{i}. {beer['name']} ({beer['rating']:.2f}★ - {beer['num_reviews']} reviews)")
                print(f"   Distance: {beer['distance']:.3f}")
        else:
            print("   No high-rated alternatives found with your criteria.")
            
        print("\n💭 Tip: The flavor combination you requested is uncommon. The alternatives above")
        print("   maintain similar characteristics but with proven appeal to beer enthusiasts.")
        
    else:
        # Good rating case - normal display
        print("━" * 60)
        print(f"✅ Great choice! Predicted rating: {predicted_rating:.2f}/5")
        print("━" * 60)
        
        print("\n🍺 Top Recommendations:")
        for i, beer in enumerate(regular_recommendations[:5], 1):
            print(f"\n{i}. {beer['name']}")
            print(f"   Rating: {beer['rating']:.2f}/5 ({beer['num_reviews']} reviews)")
            print(f"   Distance: {beer['distance']:.3f}")
            print(f"   Notes: {beer['description'][:120]}...")
    
    print("\n" + "─" * 60)


# Running Pipeline with Prompt Test Cases

print("Running Test Cases...")

test_cases = {  "I want a light citrusy beer": get_beer_features_from_text("I want a light citrusy beer"),
                "I want a strong lager which is fruity and not too malty": get_beer_features_from_text("I want a strong lager which is fruity and not too malty"),
                "I want a strong orangey tart beer": get_beer_features_from_text("I want a strong orangey tart beer"),
                "Give me a hoppy IPA with tropical notes": get_beer_features_from_text("Give me a hoppy IPA with tropical notes"),
                "I want a sessionable pilsner": get_beer_features_from_text("I want a sessionable pilsner"),
                "I need a dessert beer with chocolate and coffee notes": get_beer_features_from_text("I need a dessert beer with chocolate and coffee notes"),
                "Something light and refreshing with low alcohol": get_beer_features_from_text("Something light and refreshing with low alcohol"),
                "I want a Belgian tripel with spicy notes": get_beer_features_from_text("I want a Belgian tripel with spicy notes"),
                "Give me a bitter hoppy craft beer with no sweetness": get_beer_features_from_text("Give me a bitter hoppy craft beer with no sweetness"),
                # "I want a sweet malty amber ale": get_beer_features_from_text("I want a sweet malty amber ale"),
                # "Something sour and funky with brett character": get_beer_features_from_text("Something sour and funky with brett character"),
                # "I want a very sweet and very hoppy beer": get_beer_features_from_text(),
                "Just a Bad beer": get_beer_features_from_text("Just a Bad beer")
            }


for prompt, llm_output in test_cases.items():

    print(f"User Prompt = {prompt}")

    # PERFORMING REGRESSION
    test_point_regression = generate_test_point(llm_output, X_reg_scaled, scalar, type="Regressor")

    test_point_regression_np = test_point_regression.values[0]

    predicted_rating = gb_model.predict(test_point_regression_np.reshape(1, -1))[0]

    # Creating copies because each iteration involves subsetting and changing the Dataframes
    X_recommend_sub = X_recommend.copy()
    y_recommend_sub = y_recommend.copy()

    final_recommendations = get_beer_recommendations(llm_output, X_recommend_sub, y_recommend_sub, alt=False, alt_rating_threshold=3.0)

    alt_recommendations = None

    if predicted_rating < 3.0:
        alt_recommendations = get_beer_recommendations(llm_output, X_recommend_sub, y_recommend_sub, alt=True, alt_rating_threshold = 3.0)

    # Display results
    display_results(predicted_rating, final_recommendations, alt_recommendations)
    print('=====================================================================================================================================\n')





Running Test Cases...
User Prompt = I want a light citrusy beer
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
✅ Great choice! Predicted rating: 3.87/5
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

🍺 Top Recommendations:

1. Franziskaner Hefe-Weisse
   Rating: 4.15/5 (1528 reviews)
   Distance: 0.446
   Notes: Notes:Also known as Franziskaner Weissbier and Franziskaner Hefe-Weisse Hell.\t...

2. Blanche De Chambly
   Rating: 3.97/5 (941 reviews)
   Distance: 0.462
   Notes: Notes:The Blanche de Chambly label features the icon of the city where it is brewed: Fort Chambly. It was converted from...

3. Calabaza Blanca
   Rating: 3.99/5 (381 reviews)
   Distance: 0.483
   Notes: Notes:...

────────────────────────────────────────────────────────────

User Prompt = I want a strong lager which is fruity and not too malty
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
✅ Great choice! Predicted rating: 3.46/5
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━